# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [79]:
!cat publications.tsv

5302.24s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


pub_date	category	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2024-07-01	conferences	Priming boundaries in production: Data from French 	Proceedings of the 12th International Conference on Speech Prosody		<strong>Bevivino, D.</strong>, Huygevelde, M., Hemforth, B., &amp; Turco, G. (2024). Priming boundaries in production: Data from French. <em>Proceedings of the 12th International Conference on Speech Prosody</em>, 1005-1009. <a href="https://doi.org/10.21437/SpeechProsody.2024-203">https://doi.org/10.21437/SpeechProsody.2024-203</a>	bevivino-sp-2024		
2025-08-01	conferences	Data from online production experiments and challenges for collecting good-quality recordings for prosodic analyses 	Proceedings of the 20th International Congress of Phonetic Sciences (ICPhS 2023)		<strong>Bevivino, D</strong>., Hemforth, B., &amp; Turco, G. (2023). Data from online production experiments and challenges for collecting good-quality recordings for prosodic analyses. In: R. Skarnitzl &

## Import pandas

We are using the very handy pandas library for dataframes.

In [80]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [81]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,category,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2024-07-01,conferences,Priming boundaries in production: Data from Fr...,Proceedings of the 12th International Conferen...,NaN,"<strong>Bevivino, D.</strong>, Huygevelde, M.,...",bevivino-sp-2024,NaN,NaN
1,2025-08-01,conferences,Data from online production experiments and ch...,Proceedings of the 20th International Congress...,NaN,"<strong>Bevivino, D</strong>., Hemforth, B., &...",bevivino-icphs-2023,NaN,NaN
2,2025-10-01,inprogress,Big Team Science for language science: Opportu...,Linguistics: An Interdisciplinary Journal of t...,NaN,"Faytak, M., _. Kadavá, O. Özsoy, C. Xu, […], <...",faytak-bts-2025,NaN,NaN
3,2025-11-01,inprogress,Intonational phrase boundaries can be primed i...,in prep,NaN,"<strong>Bevivino, D.</strong>, Turco, G., &amp...",bevivino-ppen-2025,NaN,NaN
4,2025-12-01,inprogress,Priming effects of delexicalized prosodic phra...,in prep,NaN,"<strong>Bevivino, D</strong>., Hemforth, B., &...",bevivino-bap-2025,NaN,NaN
5,EXTRALINE TO RUN LAST ITEM SAFELY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [82]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [83]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"

    # MINIMAL ADDITION: add category if it exists
    if len(str(item.category)) > 0 and str(item.category) != 'nan':
        md += "\ncategory: " + str(item['category'])
    
    # close it
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

TypeError: can only concatenate str (not "float") to str

These files are in the publications directory, one directory below where we're working from.

In [ ]:
!ls ../_publications/

1113.70s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2009-10-01-paper-title-number-1.md  2025-06-08-paper-title-number-5.md
2010-10-01-paper-title-number-2.md  2025-08-01-bevivino-icphs-2023.md
2015-10-01-paper-title-number-3.md  accepted-faytak-bts-2025.md
2024-02-17-paper-title-number-4.md  inprep-bevivino-bap-2025.md
2024-07-01-bevivino-sp-2024.md	    inprep-bevivino-ppen-2025.md


In [ ]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

1118.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download slides here](http://academicpages.github.io/files/slides1.pdf)

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).